# Similarity PDF Query Tutorial

In this tutorial, we demonstrate how to load a PDF and query it.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/13-privategpt.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/13-privategpt.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/master/tutorials/13-privategpt.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

### Start EvaDB server



In [1]:
%pip install --quiet "evadb[document,notebook]"
from evadb.interfaces.relational.db import connect
conn = connect()
cursor = conn.cursor()

Note: you may need to restart the kernel to use updated packages.


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


### Download PDFs

In [2]:
!wget -nc "https://www.dropbox.com/s/6jdcn33xizdtl9k/layout-parser-paper.pdf"
!wget -nc "https://www.dropbox.com/s/4q3bvne3m2vsu5g/state_of_the_union.pdf"

File ‘layout-parser-paper.pdf’ already there; not retrieving.



File ‘state_of_the_union.pdf’ already there; not retrieving.



### Load PDFs

In [3]:
drop_pdf = cursor.query("DROP TABLE IF  EXISTS MyPDFs").execute()
load_pdf1 = cursor.load(file_regex="layout-parser-paper.pdf", format="PDF", table_name="MyPDFs").execute()
load_pdf1 = cursor.load(file_regex="state_of_the_union.pdf", format="PDF", table_name="MyPDFs").execute()

### Retrieve Text from Loaded PDFs

In [4]:
cursor.table("MyPDFs").df()

,mypdfs._row_id,mypdfs.name,mypdfs.page,mypdfs.paragraph,mypdfs.data
0,1,layout-parser-paper.pdf,1,1,LayoutParser: A Uniﬁed Toolkit for DeepLearnin...
1,1,layout-parser-paper.pdf,1,2,"Zejiang Shen1(�), Ruochen Zhang2, Melissa Dell..."
2,1,layout-parser-paper.pdf,1,3,1 Allen Institute for AIshannons@allenai.org2 ...
3,1,layout-parser-paper.pdf,1,4,Abstract. Recent advances in document image an...
4,1,layout-parser-paper.pdf,1,5,Keywords: Document Image Analysis · Deep Learn...
...,...,...,...,...,...
512,2,state_of_the_union.pdf,17,19,Now is our moment to meet and overcome the cha...
513,2,state_of_the_union.pdf,17,20,"And we will, as one people."
514,2,state_of_the_union.pdf,17,21,One America.
515,2,state_of_the_union.pdf,17,22,The United States of America.


### Create Sentence Transformer Feature Extractor UDF

In [5]:
udf_check = cursor.query(
            "DROP UDF IF  EXISTS SentenceFeatureExtractor"
        )
udf_check.execute()

# Adding Emotion detection
!wget -nc https://raw.githubusercontent.com/georgia-tech-db/eva/master/evadb/udfs/sentence_feature_extractor.py

udf = cursor.create_udf(
    "SentenceFeatureExtractor",
    True,
    "sentence_feature_extractor.py",
)
udf.execute()

06-17-2023 00:39:17 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_udf:0082] UDF SentenceFeatureExtractor does not exist, therefore cannot be dropped.


File ‘sentence_feature_extractor.py’ already there; not retrieving.



### Create vector index for paragraphs of pdf

In [6]:
cursor.create_vector_index(
            "faiss_indexs",
            table_name="MyPDFs",
            expr="SentenceFeatureExtractor(data)",
            using="QDRANT",
        ).df()

,0
0,Index faiss_indexs successfully added to the d...


In [7]:
query = (
            cursor.table("MyPDFs")
            .order(
                """Similarity(
                    SentenceFeatureExtractor('When was the NATO created?'), SentenceFeatureExtractor(data)
                )"""
            )
            .limit(3)
            .select("data")
        ).df()
query

06-17-2023 00:39:29 WARNING[vector_index_scan_executor:vector_index_scan_executor.py:exec:0086] The index faiss_indexs returned only 2 results, which is fewer than the required 3.


,mypdfs.data
0,"[38] Zhong, X., Tang, J., Yepes, A.J.: Publayn..."
1,May God bless you all. May God protect our tro...
